1. Install comyx
2. Define Simulation Parameters (num_users, channel_model, ...)
3. Generate User Profiles
4. Simulate NOMA Transmissions
5. Format the results
6. Save the dataset

In [ ]:
!pip install comyx
!pip install tqdm

#Importing Necessary Libraries

In [ ]:
from comyx.network import UserEquipment, BaseStation, Link
from comyx.propagation import get_noise_power
from comyx.utils import dbm2pow, get_distance, generate_seed, db2pow
from scipy.optimize import minimize


import numpy as np
from numba import jit
from matplotlib import pyplot as plt

#Setting Up Environment

In [ ]:
Pt = np.linspace(-10, 30, 80)  # dBm
# Pt = np.array([25]) # dBm
Pt_lin = dbm2pow(Pt)  # Watt
bandwidth = 1e7  # Bandwidth in Hz
frequency = 2.4e9  # Carrier frequency
temperature = 300  # Kelvin # 300
mc = 1  # Number of channel realizations
simulation_area_size = 60  # Size of square area (units) # 60
max_distance_bs_ue = 30     # Maximum distance from BS to each UE (units) # 30
max_distance_ue_ue = 20      # Maximum distance between UEs (units) # 20
min_distance_ue_ue = 10       # Minimum distance between UEs # 10

P_c = 1     # circuit power dBm

allocation_factors = np.linspace(0.1, 0.3, 200) # All allocation factors for UEn and UEf

N0 = get_noise_power(temperature, bandwidth)  # dBm
N0_lin = dbm2pow(N0)  # Watt
print(N0_lin)

R_prime_n = 4.0e-11
R_prime_f = 4.0e-11

n_antennas = 2

fading_args = {"type": "rayleigh", "sigma": 1 / 2}
pathloss_args = {
    "type": "reference",
    "alpha": 3.5, #3.5
    "p0": 40, # 20
    "frequency": frequency,
}  # p0 is the reference power in dBm

4.1419470000000015e-14


#Network Setup

In [ ]:
# Function to check feasibility of generated locations for UEs
def is_feasible(bs_position, ue1_position, ue2_position, size_area, max_dist_bs_ue, max_dist_ue_ue, min_dist_ue_ue):
    # Check if BS and UEs are within the area bounds
    if (abs(bs_position[0]) > size_area / 2 or abs(bs_position[1]) > size_area / 2 or
        abs(ue1_position[0]) > size_area / 2 or abs(ue1_position[1]) > size_area / 2 or
        abs(ue2_position[0]) > size_area / 2 or abs(ue2_position[1]) > size_area / 2):
        return False

    # Check distances from BS to UEs
    if (get_distance(bs_position, ue1_position) > max_dist_bs_ue or
        get_distance(bs_position, ue2_position) > max_dist_bs_ue):
        return False

    # Check distance between UEs
    if get_distance(ue1_position, ue2_position) > max_dist_ue_ue or get_distance(ue1_position, ue2_position) < min_dist_ue_ue:
        return False

    # ensure UEn is closer to base station that UEf
    if get_distance(ue1_position, bs_position) > get_distance(ue2_position, bs_position):
        # print(False)
        return False

    return True

#Initialize Links

In [ ]:
def initialize_links(BS, UEn, UEf):
  # Shapes for channels
  shape_bu = (n_antennas, n_antennas, mc)

  # Links
  # fmt: off
  link_bs_uen = Link(
      BS, UEn,
      fading_args, pathloss_args,
      shape=shape_bu, seed=generate_seed("BS-UEn"),
  )

  link_bs_uef = Link(
      BS, UEf,
      fading_args, pathloss_args,
      shape=shape_bu, seed=generate_seed("BS-UEf"),
  )

  return link_bs_uen, link_bs_uef

# Initialize Network

In [ ]:
# Initialize Setup
def initialize():
  # Initialize Base Station position
  BS = BaseStation("BS", position=[0, 0, 10], n_antennas=n_antennas, t_power=Pt_lin)

  # Loop until valid positions for UEn and UEf are found
  valid_positions_found = False
  channel_difference = False
  while not (valid_positions_found and channel_difference):
      # Generate random positions for UEn and UEf within specified constraints
      UEn_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      1]

      UEf_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      1]


      # Check if all positions are feasible
      valid_positions_found = is_feasible(BS.position, UEn_position, UEf_position,
                                          simulation_area_size, max_distance_bs_ue, max_distance_ue_ue, min_distance_ue_ue)

      # Initialize User Equipments with their feasible positions
      UEn = UserEquipment("UEn", position=UEn_position, n_antennas=n_antennas)
      UEf = UserEquipment("UEf", position=UEf_position, n_antennas=n_antennas)

      # Initialize Links
      link_bs_uen, link_bs_uef = initialize_links(BS, UEn, UEf)

      # Get channel gains
      gain_f = link_bs_uef.magnitude**2
      gain_n = link_bs_uen.magnitude**2

      # if channel difference exists
      # channel_difference =  np.all(gain_n > gain_f)
      channel_difference = np.mean(gain_n) > np.mean(gain_f)

  return BS, UEn, UEf, link_bs_uen, link_bs_uef, gain_f, gain_n

#Compute Rates & Energy Efficiency

In [ ]:
def computeEnergyEfficiency(BS, UEn, UEf, allocation_factors, gain_n, gain_f):
    results = []
    sample_index = 0

    # Looping over each allocation factor
    for alloc_UEn in allocation_factors:
        alloc_UEf = 1 - alloc_UEn

        # Update the power allocations for BS
        BS.allocations = {"UEn": alloc_UEn, "UEf": alloc_UEf}

        UEn.sinr_pre = np.zeros((len(Pt), mc))
        UEn.sinr = np.zeros((len(Pt), mc))
        UEf.sinr = np.zeros((len(Pt), mc))
        SE_nf = np.zeros((len(Pt), mc))
        SE_n = np.zeros((len(Pt), mc))
        SE_f = np.zeros((len(Pt), mc))
        SE_total = np.zeros((len(Pt), mc))
        R_n = np.zeros((len(Pt), mc))
        R_f = np.zeros((len(Pt), mc))
        EE = np.zeros((len(Pt), mc))


        # Loop over each power level
        for i, p in enumerate(Pt_lin):
            p = BS.t_power[i]

            # Loop over each channel realization
            for k in range(mc):
                gain_f_scalar = gain_f[0, 0, k]
                gain_n_scalar = gain_n[0, 0, k]

                # print(gain_f_scalar)


                # Effective transmit powers for each user
                P_n = BS.allocations["UEn"] * p  # Effective power for near user
                P_f = BS.allocations["UEf"] * p  # Effective power for far user

                # Computing Rates
                R_n[i, k] = P_n * gain_n_scalar  # Data rate for near user (received power)
                R_f[i, k] = P_f * gain_f_scalar    # Data rate for far user (received power)

                # print(R_n[i,k], R_f[i,k])

                if R_n[i, k] < R_prime_n or R_f[i, k] < R_prime_f:
                    continue  # Skip this allocation if QoS is not met


                sample_index += 1

                # Edge user
                UEf.sinr[i, k] = (BS.allocations["UEf"] * p * gain_f_scalar) / (
                    BS.allocations["UEn"] * p * gain_f_scalar + N0_lin
                )

                # Center user
                UEn.sinr_pre[i, k] = (BS.allocations["UEf"] * p * gain_n_scalar) / (
                    BS.allocations["UEn"] * p * gain_n_scalar + N0_lin
                )
                UEn.sinr[i, k] = (BS.allocations["UEn"] * p * gain_n_scalar) / N0_lin

                # Spectral Efficiencies
                SE_nf[i, k] = np.log2(1 + UEn.sinr_pre[i, k])
                SE_n[i, k] = np.log2(1 + UEn.sinr[i, k])
                SE_f[i, k] = np.log2(1 + UEf.sinr[i, k])

                # Total spectral efficiency
                SE_total[i, k] = SE_n[i, k] + SE_f[i, k]

                # Total achievable rate (bits/s)
                total_rate = R_n[i, k] + R_f[i, k]

                # Total power consumption (transmit power + circuit power)
                total_power = p + P_c

                # Energy efficiency (bits/Joule)
                EE[i, k] = SE_total[i, k] / total_power

                # print("sinr_n: ", UEn.sinr_pre[i, k])
                # print("sinr_f: ", UEf.sinr[i, k])
                # print("SE_n: ", SE_n[i, k])
                # print("total_power: ", total_power)
                # print("EE: ", EE[i, k])
                # print("alloc_UEn: ", BS.allocations["UEn"])
                # print("alloc_UEf: ", BS.allocations["UEf"])
                # print("operational power: ", Pt[i])
                # print("noise: ", N0_lin)
                # print("gain_n: ", gain_n_scalar)
                # print("gain_f: ", gain_f_scalar)
                # print("R_n: ", R_n[i, k])
                # print("R_f: ", R_f[i, k])
                # print("p_c: ", P_c)
                # print("\n")

        # Finding the optimal transmit power and channel realization for this power-allocation ensuring QoS
        max_ee_index = np.unravel_index(np.argmax(EE), EE.shape)
        optimal_power_index = max_ee_index[0]
        optimal_channel_index = max_ee_index[1]

        optimal_power = Pt[optimal_power_index]
        max_energy_efficiency = EE[max_ee_index]

        # Extract channel realization values for the optimal channel
        optimal_gain_f = gain_f[0, 0, optimal_channel_index]
        optimal_gain_n = gain_n[0, 0, optimal_channel_index]

        # Store results including allocation coefficients and max energy efficiency
        results.append({
            'sample index': sample_index,
            'alloc_UEn': alloc_UEn,
            'alloc_UEf': alloc_UEf,
            'operational_power': (optimal_power),
            "R_n": R_n[i, k],
            "R_f": R_f[i, k],
            'max_energy_efficiency': max_energy_efficiency,
            'optimal_gain_f': optimal_gain_f,
            'optimal_gain_n': optimal_gain_n,
            'optimal_channel_index': optimal_channel_index,
            'optimal_power_index': optimal_power_index,
        })

    return results

# Optimal Channel Realization

In [ ]:
def normalizeEE(results):
  min_ee = min(result['max_energy_efficiency'] for result in results)
  max_ee = max(result['max_energy_efficiency'] for result in results)

  # normalize EE
  for result in results:
      result['normalized_energy_efficiency'] = (result['max_energy_efficiency'] - min_ee) / (max_ee - min_ee) * 10

  return results

In [ ]:
def find_optimal_results(results):
  # Initialize an empty list to store the optimal results
  optimal_results = []

  # Find the entry with the maximum energy efficiency
  best_result = max(results, key=lambda x: x['max_energy_efficiency'])

  # Append the best result to the optimal_results array
  optimal_results.append(best_result)

  return optimal_results


In [ ]:
# Print the optimal results
def print_optimal_results(optimal_results):
  for res in optimal_results:
      print(f"Optimal Allocation Factors: UEn={res['alloc_UEn']:.5f}, UEf={res['alloc_UEf']:.5f}")
      print("Operational Power:", res['operational_power'])
      print("Maximum Energy Efficiency:", res['max_energy_efficiency'])
      print(f"\tNormalized Energy Efficiency: {res['normalized_energy_efficiency']:}")
      print("Optimal Channel Realization Values:")
      print("  Gain for UEf (edge user):", res['optimal_gain_f'])
      print("  Gain for UEn (center user):", res['optimal_gain_n'])

In [ ]:
# Print ALL the results
def printResults(results):
  for idx, res in enumerate(results):
      print(f"Result {idx + 1}:")
      print(f"\tAllocation Factors:")
      print(f"\t  UEn: {res['alloc_UEn']:}")
      print(f"\t  UEf: {res['alloc_UEf']:}")
      print(f"\tOperational Power: {res['operational_power']} dBm")
      print(f"\tMaximum Energy Efficiency: {res['max_energy_efficiency']:} bps/Hz")
      print(f"\tNormalized Energy Efficiency: {res['normalized_energy_efficiency']:}")
      print(f"\tOptimal Channel Realization Values:")
      print(f"\t  Gain for UEf (edge user): {res['optimal_gain_f']:}")
      print(f"\t  Gain for UEn (center user): {res['optimal_gain_n']:}")
      print("\n" + "-" * 40 + "\n")

# Running Code


In [ ]:
import json
from tqdm import tqdm

# Function to write JSON data to a single file
def write_json_file(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Initialize a list to store all runs' data
data = []

# Number of rows of data for simulation
rows_of_data = 50000

# Running for multiple network configurations
for i in tqdm(range(rows_of_data)):
    # Initialize the setup
    BS, UEn, UEf, link_bs_uen, link_bs_uef, gain_f, gain_n = initialize()

    # Compute SE for each transmit power for each channel realization for each pair of power_coefficients
    results = computeEnergyEfficiency(BS, UEn, UEf, allocation_factors, gain_n, gain_f)

    # Normalize results
    results = normalizeEE(results)

    # Find optimal results
    optimal_run_results = find_optimal_results(results)


    # Prepare data for this run
    run_data = {
        "sample index": i,
        "channel_arrays": [
            link_bs_uen.magnitude.tolist(),  # Channel matrix values for BS-UEn
            link_bs_uef.magnitude.tolist()   # Channel matrix values for BS-UEf
        ],
        "transmit_power (Watt)": BS.t_power.max(),  # Total transmit power of the BS
        "pa_factors_against_that_transmit_power": [
            res['alloc_UEn'] * BS.t_power.max() for res in optimal_run_results  # Power allocated to UEn based on max transmit power
        ],
        "optimal_results": [
            {
                "alloc_UEn": res['alloc_UEn'],
                "alloc_UEf": res['alloc_UEf'],
                "operational_power": res['operational_power'],
                "energy_efficiency": res['max_energy_efficiency'],
                "normalized_energy_efficiency": res['normalized_energy_efficiency'],
                "channel_gain_f": res['optimal_gain_f'],
                "channel_gain_n": res['optimal_gain_n']
            }
            for res in optimal_run_results  # Loop through optimal results
        ]
    }

    # Append this run's data to the list of all runs' data
    data.append(run_data)

    # print()
    # print_optimal_results(optimal_run_results)

# Write all runs' data to a single JSON file at the end of all simulations
write_json_file(data, 'EE_data_50K_1.json')


  0%|          | 12/50000 [00:02<1:58:01,  7.06it/s]<ipython-input-25-3dcd96b06456>:7: RuntimeWarning: invalid value encountered in scalar divide
  result['normalized_energy_efficiency'] = (result['max_energy_efficiency'] - min_ee) / (max_ee - min_ee) * 10
100%|██████████| 50000/50000 [1:07:20<00:00, 12.38it/s]


In [ ]:
# Download json file
from google.colab import files
files.download('EE_data_50K_1.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Archive

In [ ]:
# import numpy as np

# def calculate_EE(p_c_lin, N0_lin, alloc_UEn, alloc_UEf, operational_power, channel_gain_f, channel_gain_n):
#     """
#     Calculate Energy Efficiency (EE) given the provided parameters.

#     Parameters:
#         p_c_lin (float): Circuit power in linear scale (Watts).
#         N0_lin (float): Noise power in linear scale.
#         alloc_UEn (float): Power allocation factor for the near user.
#         alloc_UEf (float): Power allocation factor for the far user.
#         operational_power (float): Transmit power in dBm (converted to linear scale inside the function).
#         channel_gain_f (float): Channel gain for the far user.
#         channel_gain_n (float): Channel gain for the near user.

#     Returns:
#         float: Energy Efficiency (EE) in bits/Joule.
#     """
#     # Convert dBm to linear scale for operational power
#     p_tx = 10 ** (operational_power / 10) / 1000  # Convert dBm to Watts

#     # Effective transmit powers for each user
#     P_n = alloc_UEn * p_tx  # Effective power for near user
#     P_f = alloc_UEf * p_tx  # Effective power for far user

#     # Compute SINR for near and far users
#     sinr_f = (alloc_UEf * p_tx * channel_gain_f) / (alloc_UEn * p_tx * channel_gain_f + N0_lin)
#     sinr_n = (alloc_UEn * p_tx * channel_gain_n) / N0_lin

#     # Compute Spectral Efficiencies
#     SE_f = np.log2(1 + sinr_f)  # Far user spectral efficiency
#     SE_n = np.log2(1 + sinr_n)  # Near user spectral efficiency

#     # Total spectral efficiency
#     SE_total = SE_n + SE_f

#     # Total power consumption (transmit power + circuit power)
#     total_power = p_tx + p_c_lin

#     # Energy Efficiency (bits/Joule)
#     EE = SE_total / total_power

#     return EE

In [ ]:
# calculate_EE(1.0, 4.1419470000000015e-14
# , 0.13115577889447236, 0.8688442211055276, 23.417721518987342, 2.0960336981693498e-10, 8.9069054859933e-09)

In [ ]:
# # Fixing sample indices

# import json

# # Load the JSON file
# def load_json(file_path):
#     with open(file_path, 'r') as file:
#         return json.load(file)

# # Update the "sample index" field
# def update_sample_index(data):
#     for i, entry in enumerate(data):
#         entry['sample index'] = i + 1

# # Save the updated JSON data
# def save_json(data, output_path):
#     with open(output_path, 'w') as file:
#         json.dump(data, file, indent=4)

# # Main function
# def main():
#     # Load the JSON data
#     original_dataset = load_json("filtered_150K.json")

#     # Update the "sample index" field
#     update_sample_index(original_dataset)

#     # Save the updated JSON data
#     save_json(original_dataset, "150K_fixed.json")

#     print("JSON file updated successfully.")

# # Run the main function
# main()


In [ ]:
# import json


# # Count objects with "energy_efficiency" = 0.0
# def count_zero_energy_efficiency(data):
#     filtered_data = [
#         entry for entry in data
#         if entry.get('optimal_results', [{}])[0].get('energy_efficiency', None) != 0.0
#     ]
#     return filtered_data

# # Save the updated JSON data
# def save_json(data, output_path):
#     with open(output_path, 'w') as file:
#         json.dump(data, file, indent=4)

# # Main function
# def main():
#     original_dataset = load_json("EE_150K.json")

#     # Filter out entries with "energy_efficiency" = 0.0
#     filtered_data = count_zero_energy_efficiency(original_dataset)

#     # Save the updated JSON data
#     output_file_name = "filtered_150K.json"
#     save_json(filtered_data, output_file_name)

# # Run the main function
# main()
